# Project Data Acquisition

This code downloads the [bchydro-outages](https://github.com/outages/bchydro-outages/tree/main) project (including all commit history) from GitHub and saves it to a sub directory.

Then it does some work to process it into a usable Pandas-compatible format.

## Configuration

In [ ]:
# Be wary of how much compute big numbers require
# 14 days took ~30 seconds to run on my machine, plus 15 seconds to download the repo - Bea
DAYS_TO_CAPTURE = 3
"""The number of days of history to capture in the returned Pandas Data"""

DELETE_OLD_REPO = False
"""If True, deletes the old repository data and starts fresh"""

'If True, deletes the old repository data and starts fresh'

## Fetch the Repository

In [124]:
import subprocess
import os
import shutil

target = os.getcwd()
repoName = "bchydro-outages"

repoPath = os.path.join(target, repoName)

if DELETE_OLD_REPO and os.path.exists(repoPath):
  # https://stackoverflow.com/a/6996628 <-- How to delete a directory in Python
  shutil.rmtree(repoPath)

# https://stackoverflow.com/a/4760517 <-- How to run subprocess in Python

# Clone the REPO and save to <repoName> folder under DataAcquisition/
result = subprocess.run(["git", "clone", "https://github.com/outages/bchydro-outages.git", f"./{repoName}"], cwd=target, capture_output=True)
print(result.stdout.decode("utf-8"))
print(result.stderr.decode("utf-8"))

# Confirm that the repository was cloned
assert os.path.exists(repoPath)


Cloning into './bchydro-outages'...



# Process commit history

The repo only has 1 file in it: a `.json` file which shows the current (right now) outages being tracked by BC Hydro. To find historical data, we need to traverse the commit history and merge each commit together


Step 1) Get the JSON data from each commit

In [125]:
from datetime import datetime
import json

# Reset to main branch
subprocess.run(["git", "checkout", "main"], cwd=repoPath)

# Find the oldest commit from N days ago
afterTime = f"{DAYS_TO_CAPTURE} days ago"

commitLog = subprocess.run(["git", "log", f"--since={afterTime}", "--pretty=format:%H:%ct", "--reverse"], cwd=repoPath, capture_output=True)
commits = commitLog.stdout.decode("utf-8")

commits = [commit.split(":") for commit in commits.split("\n")]

# Get JSON file for each commit

OUTAGES_FILE_NAME = "bchydro-outages.json"
outagesFilePath = os.path.join(repoPath, OUTAGES_FILE_NAME)

def getJSON(commit):
  subprocess.run(["git", "checkout", commit[0]], cwd=repoPath, capture_output=True)
  assert os.path.exists(os.path.join(repoPath, outagesFilePath))
  with open(outagesFilePath, "r") as f:
    # https://stackoverflow.com/q/20199126 <-- How to load JSON from a file
    return json.load(f)

jsonData = [getJSON(commit) for commit in commits]

jsonData[0]

[{'id': 2555322,
  'gisId': 2528863,
  'regionId': 521980323,
  'municipality': 'West Vancouver',
  'area': '1100 block MILLSTREAM RD',
  'cause': 'Tree down across our wires',
  'numCustomersOut': 4,
  'crewStatus': 'ONSITE',
  'crewStatusDescription': 'Crew on-site',
  'crewStatusNote': '',
  'crewEta': 1740892800000,
  'dateOff': 1740889680000,
  'dateOn': 1740904200000,
  'lastUpdated': 1740899510000,
  'regionName': 'Lower Mainland',
  'crewEtr': 1740904200000,
  'showEta': False,
  'showEtr': True,
  'latitude': 49.355851,
  'longitude': -123.149873,
  'polygon': [-123.147945,
   49.3565,
   -123.147971,
   49.356672,
   -123.148047,
   49.356838,
   -123.148171,
   49.356992,
   -123.148337,
   49.357127,
   -123.148541,
   49.357238,
   -123.148773,
   49.357322,
   -123.149025,
   49.357376,
   -123.149288,
   49.357396,
   -123.149553,
   49.357384,
   -123.149809,
   49.357338,
   -123.150047,
   49.357261,
   -123.150258,
   49.357156,
   -123.150433,
   49.357026,
   -123.

Step 2) Combine the JSON data

Because this tracks current outages, we never actually get to see the "timeOn" at the end. The `dateOn` field in the JSON is only an estimate time. 

But, we can interpret when the outage ended (within +-15 minutes) by seeing if it is present in the next commit. This is why we'll start from the latest commit and work towards present-day

This does mean that the most recent outages (any that are still ongoing) won't be added to the list, but that's fine

In [126]:
# Make sure to take all but start time from the latest entry for an outage

import pandas as pd

outages = []


waitingForEndTime = []
for i, data in enumerate(jsonData):
  commitTime = commits[i][1]

  # data is a list of active outages
  activeOutageIds = [outage["id"] for outage in data]

  # Check if any of the outages in waitingForEndTime are in the active outages
  # Push to final array if they aren't

  newWaiting = []
  for waitingOutage in waitingForEndTime:
    if waitingOutage["id"] not in activeOutageIds:
      waitingOutage["endTime"] = commitTime
      outages.append(waitingOutage)
    else:
      newWaiting.append(waitingOutage)
  waitingForEndTime = newWaiting

  # Override any outages waiting for time with latest data
  for i, waitingOutage in enumerate(waitingForEndTime):
    for outage in data:
      if waitingOutage["id"] == outage["id"]:
        waitingForEndTime[i] = outage
        break

  # Add any new outages to waitingForEndTime
  for outage in data:
    if outage["id"] not in [waitingOutage["id"] for waitingOutage in waitingForEndTime]:
      waitingForEndTime.append(outage)

print(f"Outages Indexed: {len(outages)}")
print(f"Active Outages: {len(waitingForEndTime)}")

frame = pd.DataFrame(outages).sort_values(by="endTime")

frame[frame['municipality'] == 'Vancouver']

Outages Indexed: 878
Active Outages: 5


,id,gisId,regionId,municipality,area,cause,numCustomersOut,crewStatus,crewStatusDescription,crewStatusNote,crewEta,dateOff,dateOn,lastUpdated,regionName,crewEtr,showEta,showEtr,latitude,longitude,polygon,endTime
9,2555494,2529035,521980323,Vancouver,"South of 5TH AVE, East of BAYSWATER ST, North ...",Planned work being done on our equipment,113,ONSITE,Crew on-site,,1.740932e+12,1740932260000,1.740964e+12,1740932616000,Lower Mainland,1.740964e+12,False,True,49.266019,-123.168258,"[-123.165298, 49.265183, -123.165318, 49.26533...",1740946230
18,2555521,2529062,521980323,Vancouver,100 block W 5TH AVE,Equipment failure,6,ONSITE,Crew on-site,,1.740954e+12,1740953500000,1.740958e+12,1740953695000,Lower Mainland,1.740958e+12,False,True,49.266878,-123.107933,"[-123.105769, 49.266845, -123.105794, 49.26701...",1740955047
28,2555489,2529030,521980323,Vancouver,"900 block E HASTINGS ST, 300 block RAYMUR AVE",Planned work being done on our equipment,375,ONSITE,Crew on-site,,1.740932e+12,1740931332000,1.741000e+12,1740983644000,Lower Mainland,1.741000e+12,False,True,49.281332,-123.083932,"[-123.083931, 49.280433, -123.083663, 49.28045...",1740995224
29,2555490,2529031,521980323,Vancouver,100 block N COMMERCIAL DR,Planned work being done on our equipment,2,ONSITE,Crew on-site,,1.740932e+12,1740931437000,1.741000e+12,1740983644000,Lower Mainland,1.741000e+12,False,True,49.284702,-123.071672,"[-123.07167, 49.283803, -123.071402, 49.283821...",1740995224
45,2555593,2529132,521980323,Vancouver,"4100 - 4200 block ATLIN ST, 4100 - 4200 block ...",Planned work being done on our equipment,19,ONSITE,Crew on-site,,1.741019e+12,1741018924000,1.741050e+12,1741020676000,Lower Mainland,1.741050e+12,False,True,49.246833,-123.046506,"[-123.044954, 49.247185, -123.044981, 49.24736...",1741036190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810,2559699,2533185,521980323,Vancouver,"West of TRAFALGAR ST, North of 32ND AVE, South...",Planned work being done on our equipment,10,ONSITE,Crew on-site,,1.741968e+12,1741967574000,1.741991e+12,1741977115000,Lower Mainland,1.741991e+12,False,True,49.243719,-123.167426,"[-123.165784, 49.243652, -123.1658, 49.243787,...",1741987601
814,2559687,2533173,521980323,Vancouver,"8400 - 8500 block OAK ST, 8400 - 8500 block SH...",Planned work being done on our equipment,11,ONSITE,Crew on-site,,1.741966e+12,1741965793000,1.741990e+12,1741981075000,Lower Mainland,1.741990e+12,False,True,49.209224,-123.129514,"[-123.127972, 49.209063, -123.127975, 49.20933...",1741988829
815,2559703,2533189,521980323,Vancouver,"North of 54TH AVE, West of LAUREL ST, South of...",Planned work being done on our equipment,30,ONSITE,Crew on-site,,1.741969e+12,1741968188000,1.741997e+12,1741968831000,Lower Mainland,1.741997e+12,False,True,49.222675,-123.126041,"[-123.124469, 49.22358, -123.12447, 49.223609,...",1741988829
830,2559842,2533324,521980323,Vancouver,"500 block MCLEAN DR, 1400 block FRANCES ST",Planned work being done on our equipment,12,ONSITE,Crew on-site,,1.741992e+12,1741991597000,1.741995e+12,1741992785000,Lower Mainland,1.741995e+12,False,True,49.279831,-123.074779,"[-123.074778, 49.278932, -123.07451, 49.27895,...",1741999942
